In [158]:
%reset -f
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [159]:
CSV_file = "SDS00005.CSV"

df = pd.read_csv(CSV_file, skiprows=[1])
df.describe()

Rhi = 4.7e6*5
Rlo = 110e3
G = (Rhi+Rlo)/Rlo
G *= 1.05

# Assign decent names to the columns and compute derivatives
df["t"] = df.Source - df.Source.min()
if (CSV_file == "SDS00001.CSV"):
    df["u_in_gt"] = df.CH4
    df["u_out"] = df.CH1 * G
    del df["CH1"]
    del df["CH4"]
    begin_at_value = +1000
    end_at_value = -1000
elif (CSV_file == "SDS00002.CSV"):
    df["u_in_gt"] = df.CH4
    df["u_out"] = df.CH1 * G
    del df["CH1"]
    del df["CH4"]
    begin_at_value = +1000
    end_at_value = -20
elif (CSV_file == "SDS00003.CSV"):
    df["u_in_gt"] = df.CH1
    df["u_out"] = df.CH3 * G
    del df["CH1"]
    del df["CH3"]
    begin_at_value = +1000
    end_at_value = -20
elif (CSV_file == "SDS00004.CSV"):
    df["u_in_gt"] = df.CH1
    df["u_out"] = df.CH3 * G
    del df["CH1"]
    del df["CH3"]
    begin_at_value = +1000
    end_at_value = -21
elif (CSV_file == "SDS00005.CSV"):
    df["u_in_gt"] = df.CH4
    df["u_out"] = df.CH3 * G
    del df["CH4"]
    del df["CH3"]
    begin_at_value = +1000
    end_at_value = -20
# Remove the silly-named original columns
del df["Source"]


desired_sample_rate = 1e3
time_span = df.t.max() - df.t.min()
desired_num_samples = int(time_span * desired_sample_rate)
downsample_factor = round(len(df) / desired_num_samples)
print(f"{downsample_factor=}")
num_rows = (len(df) // downsample_factor) * downsample_factor
df = df.iloc[:num_rows]
reshaped_df = df.values.reshape(-1, downsample_factor, df.shape[1])
downsampled_data = reshaped_df.mean(axis=1)
dsdf = pd.DataFrame(downsampled_data, columns=df.columns)
dsdf["dt"] = dsdf.t.diff()
dsdf["du_out_dt"] = dsdf.u_out.diff() / dsdf.dt
dsdf["du_out_dt_lpf"] = dsdf.du_out_dt.rolling(5).mean()

# Trim the dataframe to only include the part where the cap is being charged
begin_at = dsdf[dsdf.du_out_dt_lpf > begin_at_value].index[0] - 1
dsdf     = dsdf.iloc[begin_at:]
end_at   = dsdf[dsdf.du_out_dt_lpf < end_at_value].index[0] - 1
dsdf     = dsdf.iloc[:end_at-begin_at]
dsdf

# Trim the time base.
dsdf["t"] = dsdf.t - dsdf.t.min()
dsdf.describe()

downsample_factor=500


,t,u_in_gt,u_out,dt,du_out_dt,du_out_dt_lpf
count,831.000000,831.000000,831.000000,8.310000e+02,831.000000,831.000000
mean,0.415000,516.784019,512.171112,1.000000e-03,839.465289,842.073160
std,0.240033,177.863093,174.441991,1.982648e-16,973.080881,848.982732
min,0.000000,2.660000,3.389537,1.000000e-03,-901.472727,-7.211782
25%,0.207500,422.070000,420.627175,1.000000e-03,234.382909,306.500727
50%,0.415000,574.060000,569.550469,1.000000e-03,631.030909,522.854182
75%,0.622500,659.140000,650.259322,1.000000e-03,1207.973455,1061.934873
max,0.830000,708.700000,699.092100,1.000000e-03,6724.986545,4788.623127


In [160]:
R_eq = (Rhi * Rlo) / (Rhi + Rlo)
C_out = 1e-9
tau_init = C_out * R_eq
tau_init

0.0001094875052943668

In [161]:
def predict_u_in(u_out, du_out_dt, tau):
    return u_out + tau * du_out_dt

dsdf["u_in_pred"] = predict_u_in(dsdf.u_out, dsdf.du_out_dt_lpf, tau_init)

fig = go.Figure()
fig.update_layout(
    height=400,
    hovermode='x',
    yaxis2=dict(title='u_c [V]', overlaying='y', side='right'),
)
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_in_gt, name="u_in_gt"))
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_out, name="u_out"))
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_in_pred, name="u_in_pred"))
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.du_out_dt_lpf, name="du_out/dt", yaxis='y2'))

In [162]:
fig = go.Figure()
fig.update_layout(height=400, hovermode='x')
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_in_pred-dsdf.u_in_gt, name="u_err"))